In [0]:
%tensorflow_version 1.x

In [3]:
!pip3 install kashgari

     |████████████████████████████████| 81kB 1.5MB/s 
     |████████████████████████████████| 17.3MB 491kB/s 
     |████████████████████████████████| 645kB 22.2MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=1dadc1fef0e66bed7c8a03311ecc0cccf00468e0508048c75a40770e63b96aec
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=4e03f5d024ff91aee4b70078e196717eb9c5fe76a8612261f762d341a8196d2c
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=95abbf21c2b6ec7e033cab8ea5c5e54537ac5a30bbb27a6850bd9dbc342e3e13
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel f

In [4]:
# 下载数据集
!mkdir -p ~/datasets/
!wget "https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv" -O ~/datasets/data.csv
!ls ~/datasets

--2019-11-04 03:00:32--  https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2904858 (2.8M) [text/plain]
Saving to: ‘/root/datasets/data.csv’

/root/datasets/data 100%[===================>]   2.77M  16.8MB/s    in 0.2s    

2019-11-04 03:00:33 (16.8 MB/s) - ‘/root/datasets/data.csv’ saved [2904858/2904858]

data.csv


In [5]:
import kashgari
import jieba
import pandas as pd
from kashgari.tasks.classification import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

import logging
logging.basicConfig(level = "DEBUG")

In [0]:
data_file_path = "~/datasets/data.csv"

data_frame = pd.read_csv(data_file_path, sep=",")

In [7]:
data_frame.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [0]:
data_frame["cutted"] = data_frame["review"].apply(lambda x : list(jieba.cut(str(x))))

In [10]:
data_frame.head()

,label,review,cutted
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙, 公路, 较近, ,, 但是, 公交, 指示, 不, 对, ,, 如果, 是..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大床, 房, ，, 房间, 很大, ，, 床有, 2M, 宽, ，, 整体, 感觉..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太, 差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, ..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小, 街道, 上, ，, 不大好, 找, ，, 但, 还好, 北京, 热心,..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD, 中心, ,, 周围, 没什么, 店铺, ,, 说, 5, 星, 有点, 勉强, ..."


In [17]:
# 平衡数据
df_positive = data_frame[data_frame.label == 1]
df_negative = data_frame[data_frame.label == 0]

half_size = len(df_positive) if len(df_positive) < len(df_negative) else len(df_negative)

df_balanced = pd.concat([df_positive.sample(half_size, replace=df_positive.shape[0] < half_size), df_negative.sample(half_size, replace=df_negative.shape[0] < half_size)])

print(len(df_balanced[df_balanced.label == 1]), len(df_balanced[df_balanced.label == 0]))

2444 2444


In [22]:
df_balanced.sample(5)

,label,review,cutted
7384,0,住的标准单人间实在是小，房间里一张大床已经塞得满满的了，设施极其简陋除了电视机和烧水壶其他什...,"[住, 的, 标准, 单人间, 实在, 是, 小, ，, 房间, 里, 一张, 大床, 已经..."
6538,0,作为这个价位的经济型酒店，总体还算可以。但是房间构局极不合理，一个小桌子紧挨着床，上网必须盘...,"[作为, 这个, 价位, 的, 经济型, 酒店, ，, 总体, 还, 算, 可以, 。, 但..."
5932,0,"大厅雄伟,但前台和房间太小、(走廊太长)房间分布太散，找同伴很累。如果能将宾馆分为2个馆,前...","[大厅, 雄伟, ,, 但, 前台, 和, 房间, 太小, 、, (, 走廊, 太, 长, ..."
3739,1,到达绍兴后，只要给这个客栈一个电话，十分钟之内就会有一辆小面的来接你，很方便。“老台门客栈”...,"[到达, 绍兴, 后, ，, 只要, 给, 这个, 客栈, 一个, 电话, ，, 十分钟, ..."
6360,0,前台接待态度生硬，楼道内因很多剧组，脏乱不堪。房间内部面积很大（大床房），但是家具很少，而且...,"[前台, 接待, 态度生硬, ，, 楼道, 内因, 很多, 剧组, ，, 脏乱, 不堪, 。..."


In [23]:
df_balanced.shape

(4888, 3)

In [37]:
total = df_balanced.shape[0]

print("total = {}".format(total))

random_frame = df_balanced.sample(df_balanced.shape[0])

random_frame.head()


total = 4888


,label,review,cutted
3030,1,我的国外客人十分满意，我本人对大堂和门童也是很满意。,"[我, 的, 国外, 客人, 十分满意, ，, 我, 本人, 对, 大堂, 和, 门童, 也..."
7074,0,"提早预定的房间3间标房4夜，到了酒店,看样子是个刚改造的样子,很多地方还留着小旅馆的舍不得仍...","[提早, 预定, 的, 房间, 3, 间, 标房, 4, 夜, ，, 到, 了, 酒店, ,..."
5551,0,硬件还算过得去。就是服务水准太差了。前台服务员好像都是煞笔，一个个装傻充愣，一问三不知！！另...,"[硬件, 还, 算, 过得去, 。, 就是, 服务, 水准, 太差, 了, 。, 前台, 服..."
6602,0,因为到达时间有点晚，所以携程的房间取消了。只好拿了2个套房和4个单间。房间是日式的，面积确实...,"[因为, 到达, 时间, 有点, 晚, ，, 所以, 携程, 的, 房间, 取消, 了, 。..."
1287,1,如果中餐再丰富一些就好了，一个人吃中餐没有找到合适的地方。西餐不错,"[如果, 中餐, 再, 丰富, 一些, 就, 好, 了, ，, 一个, 人, 吃, 中餐, ..."


In [0]:
split_point = [total * 70 // 100, total * 85 // 100]

train_x = list(random_frame["cutted"][:split_point[0]])
train_y = list(random_frame["label"][:split_point[0]])

valid_x = list(random_frame["cutted"][split_point[0]: split_point[1]])
valid_y = list(random_frame["label"][split_point[0]: split_point[1]])

test_x = list(random_frame["cutted"][split_point[1]:])
test_y = list(random_frame["label"][split_point[1]:])

In [12]:
# 下载 bert 预训练模型
!mkdir -p ~/bert_models
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip -O ~/bert_models/chinese_L-12_H-768_A-12.zip
!cd ~/bert_models && unzip chinese_L-12_H-768_A-12.zip

--2019-11-04 03:05:29--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘/root/bert_models/chinese_L-12_H-768_A-12.zip’

/root/bert_models/c 100%[===================>] 364.20M  78.0MB/s    in 4.7s    

2019-11-04 03:05:34 (78.0 MB/s) - ‘/root/bert_models/chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-

In [39]:
bert_embedding = BERTEmbedding("/root/bert_models/chinese_L-12_H-768_A-12",
                               task=kashgari.CLASSIFICATION,
                               sequence_length=64)

In [47]:
model = BiLSTM_Model(bert_embedding)
model.fit(train_x, train_y, valid_x, valid_y, batch_size=1024, epochs=5)

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 64, 768), (2 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 64, 768)      1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [50]:
model.evaluate(test_x, test_y, batch_size=512)

              precision    recall  f1-score   support

           0     0.6543    0.4889    0.5596       360
           1     0.6043    0.7513    0.6698       374

    accuracy                         0.6226       734
   macro avg     0.6293    0.6201    0.6147       734
weighted avg     0.6288    0.6226    0.6158       734

